FaceParsing
====

```bash
pip install torch torchvision
pip install transformers
```

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch import nn
from transformers import (
    SegformerImageProcessor,
    SegformerForSemanticSegmentation
)

device = ("cuda" if torch.cuda.is_available() else "cpu")

# load models
image_processor = SegformerImageProcessor.from_pretrained(
    "jonathandinu/face-parsing"
)
model = SegformerForSemanticSegmentation.from_pretrained(
    "jonathandinu/face-parsing"
)
model.eval().to(device);